Gradient Boosting Machines (GBMs) with neural network backbones

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Necessary Libraries

In [2]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

Load and Prepare the Dataset

In [4]:
# # Load your dataset
# clean_df = pd.read_csv('clean_df.csv')
# clean_df.head(5)

file_path = '/content/drive/MyDrive/DU_AI_Bootcamp/23_FinalProject/House_of_Hope/clean_df.csv'
clean_df = pd.read_csv(file_path)
clean_df.head()

# Separate features and target
X = clean_df.drop('REASON', axis=1)
y = clean_df['REASON']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Train Gradient Boosting Machine

In [5]:
# Initialize and train Gradient Boosting Classifier
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm.fit(X_train, y_train)

# Make predictions with GBM
gbm_train_preds = gbm.predict_proba(X_train)[:, 1]
gbm_test_preds = gbm.predict_proba(X_test)[:, 1]

Prepare Data for Neural Network

In [6]:
# Combine original features with GBM predictions
X_train_nn = np.hstack((X_train, gbm_train_preds.reshape(-1, 1)))
X_test_nn = np.hstack((X_test, gbm_test_preds.reshape(-1, 1)))

In [7]:
# One-hot encode the target variable
y_train_nn = to_categorical(y_train)
y_test_nn = to_categorical(y_test)

Build and Train the Neural Network

In [8]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_nn.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Assuming binary classification

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_nn, y_train_nn, epochs=10, batch_size=32, validation_data=(X_test_nn, y_test_nn))


Epoch 1/10
161037/161037 [==============================] - 314s 2ms/step - loss: 0.4180 - accuracy: 0.7958 - val_loss: 0.4031 - val_accuracy: 0.8046
Epoch 2/10
161037/161037 [==============================] - 310s 2ms/step - loss: 0.4013 - accuracy: 0.8058 - val_loss: 0.3983 - val_accuracy: 0.8070
Epoch 3/10
161037/161037 [==============================] - 312s 2ms/step - loss: 0.3972 - accuracy: 0.8085 - val_loss: 0.3956 - val_accuracy: 0.8103
Epoch 4/10
161037/161037 [==============================] - 305s 2ms/step - loss: 0.3949 - accuracy: 0.8101 - val_loss: 0.3949 - val_accuracy: 0.8099
Epoch 5/10
161037/161037 [==============================] - 302s 2ms/step - loss: 0.3934 - accuracy: 0.8110 - val_loss: 0.3928 - val_accuracy: 0.8114
Epoch 6/10
161037/161037 [==============================] - 300s 2ms/step - loss: 0.3924 - accuracy: 0.8116 - val_loss: 0.3913 - val_accuracy: 0.8121
Epoch 7/10
161037/161037 [==============================] - 302s 2ms/step - loss: 0.3916 - accuracy:

Evaluate the Model

In [9]:
# Evaluate the model on the test data
y_pred_nn = model.predict(X_test_nn)
y_pred_nn_classes = np.argmax(y_pred_nn, axis=1)

40260/40260 [==============================] - 49s 1ms/step


Calculate Accuracy

In [10]:
from sklearn.metrics import accuracy_score
# Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred_nn)
# print('Accuracy:', accuracy)
# Assuming y_test needs to be converted to binary format
y_test_binary = np.where(y_test > 0.5, 1, 0) # Convert probabilities to binary (adjust threshold if needed)

# Calculate accuracy using the binary version of y_test
accuracy = accuracy_score(y_test_binary, y_pred_nn_classes)
print('Accuracy:', accuracy)

Accuracy: 0.8126561173148366


In [11]:
# Classification report and ROC AUC score
print(classification_report(y_test, y_pred_nn_classes))
print('ROC AUC Score:', roc_auc_score(y_test, y_pred_nn[:, 1]))

              precision    recall  f1-score   support

           0       0.78      0.68      0.73    472472
           1       0.83      0.89      0.86    815822

    accuracy                           0.81   1288294
   macro avg       0.80      0.78      0.79   1288294
weighted avg       0.81      0.81      0.81   1288294

ROC AUC Score: 0.8923675717108549


Save the GBM Model

In [14]:
# Save the model
joblib.dump(gbm, '/content/drive/MyDrive/DU_AI_Bootcamp/23_FinalProject/House_of_Hope/gbm_model_classified_crossentropy.pkl')

['/content/drive/MyDrive/DU_AI_Bootcamp/23_FinalProject/House_of_Hope/gbm_model_classified_crossentropy.pkl']

Load the Saved GBM Model

In [15]:
# Load the model
gbm_loaded = joblib.load('/content/drive/MyDrive/DU_AI_Bootcamp/23_FinalProject/House_of_Hope/gbm_model_classified_crossentropy.pkl')

# Verify the loaded model
print(gbm_loaded.predict(X_test[:5]))  # Predict using the loaded model to verify

[0 1 0 0 1]


Summary


*   Load and preprocess the data: Normalize features and split the dataset.
*   Train a Gradient Boosting Machine (GBM): Use the GBM to predict probabilities.
*   Prepare data for the neural network: Combine the original features with GBM predictions and one-hot encode the target.
*   Build and train a neural network: Use Keras to define, compile, and train the neural network.
*   Evaluate the model: Assess the performance using classification metrics and ROC AUC score.

This approach leverages the strengths of both GBMs and neural networks, potentially leading to better performance on complex datasets.